In [79]:
import numpy as np
import pandas as pd
import seaborn as sns


from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [80]:
import matplotlib.pyplot as plt
from matplotlib import rc
from pylab import rcParams
from datetime import datetime
import matplotlib.pyplot as plt 
!pip install --quiet pytorch-lightning==1.2.5
#progress bars and much more
#TODO: read documentation
!pip install --quiet tqdm==4.59.0

from tqdm.notebook import tqdm
import pytorch_lightning as pl
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


In [81]:
rcParams['figure.figsize'] = 13,7

tqdm.pandas()

pl.seed_everything(7)

df = pd.read_csv('/content/drive/MyDrive/allwind.csv')
df = df.drop(['ISTANBUL WindSpeed(m/s)'], axis = 1)
df = df.drop(['Unnamed: 0'] , axis = 1 )

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  def get_lock(cls):
Global seed set to 7


In [82]:
df['Date'] = pd.date_range(start = '20180201' , freq = 'H' , periods = len(df))
df.dropna(inplace = True)
df.head()

,target_load,AMASYA WindSpeed(m/s),KIRLARELI WindSpeed(m/s),TEKIRDAG WindSpeed(m/s),EDIRNE WindSpeed(m/s),BURSA WindSpeed(m/s),SIVAS WindSpeed(m/s),BILECIK WindSpeed(m/s),AYDIN WindSpeed(m/s),HATAY WindSpeed(m/s),TOKAT WindSpeed(m/s),KONYA WindSpeed(m/s),GAZIANTEP WindSpeed(m/s),OSMANIYE WindSpeed(m/s),CNK WindSpeed(m/s),MUGLA WindSpeed(m/s),YALOVA WindSpeed(m/s),ADIYAMAN WindSpeed(m/s),KOC WindSpeed(m/s),KAYSERI WindSpeed(m/s),ISPARTA WindSpeed(m/s),USAK WindSpeed(m/s),MERSIN WindSpeed(m/s),IZMIR WindSpeed(m/s),MANISA WindSpeed(m/s),AFYON WindSpeed(m/s),Date
0,49.25,5.175035,3.737460,5.232393,7.066192,0.000000,6.420591,3.083416,5.449154,0.0,2.955127,4.978358,0.0,0.0,0.000000,0.000000,2.593469,0.000000,4.171543,0.0,0.000000,2.652624,5.281608,0.000000,3.320128,1.028135,2018-02-01 00:00:00
1,658.52,5.175035,5.211629,6.348752,7.404708,4.937590,7.183495,0.000000,6.353604,0.0,3.684031,4.980009,0.0,0.0,1.661267,1.172743,3.740429,0.000000,3.335302,0.0,1.065564,2.397490,5.787553,2.845727,4.239942,1.395394,2018-02-01 01:00:00
2,0.00,5.175035,6.232947,7.267507,7.958284,6.789526,6.986767,1.829235,8.292643,0.0,4.591133,4.730059,0.0,0.0,3.235681,3.199151,4.434769,0.000000,5.063971,0.0,2.234558,0.000000,5.372688,4.866125,5.598036,1.918763,2018-02-01 02:00:00
3,-788.52,5.175035,7.760160,8.973306,7.823372,6.220973,4.790768,2.698536,8.849569,0.0,5.501732,4.219697,0.0,0.0,4.567084,5.774808,5.186938,0.000000,5.488671,0.0,2.659959,2.763606,5.787553,5.171029,6.750460,2.251368,2018-02-01 03:00:00
4,-497.19,5.923936,9.088173,9.807333,9.404032,7.705063,4.527996,3.567999,7.601854,0.0,5.739936,4.156707,0.0,0.0,2.685054,7.068883,6.920417,2.108324,6.464936,0.0,1.065564,3.740429,6.150193,5.919351,8.038033,2.790788,2018-02-01 04:00:00


In [83]:
# preprocessing:
rowsData = []
for i,row in tqdm(df.iterrows() , total = len(df)):

  data = dict(
      target = row.target_load,
      weekDay = row.Date.dayofweek,
      weekOfYear = row.Date.week,
      hourOfDay = row.Date.hour,
      dayOfMonth = row.Date.day,
      AMASYA_WindSpeed = row['AMASYA WindSpeed(m/s)'],
      KIRLARELIWindSpeed = row['KIRLARELI WindSpeed(m/s)'],
      TEKIRDAGWindSpeed = row['TEKIRDAG WindSpeed(m/s)'],
      EDIRNEWindSpeed = row['EDIRNE WindSpeed(m/s)'],
      BURSAWindSpeed = row['BURSA WindSpeed(m/s)'],
      SIVASWindSpeed = row['SIVAS WindSpeed(m/s)'],
      BILECIKWindSpeed = row['BILECIK WindSpeed(m/s)'],
      AYDINWindSpeed = row['AYDIN WindSpeed(m/s)'],
      HATAYWindSpeed = row['HATAY WindSpeed(m/s)'],
      TOKATWindSpeed = row['TOKAT WindSpeed(m/s)'],
      KONYAWindSpeed = row['KONYA WindSpeed(m/s)'],
      GAZIANTEPWindSpeed = row['GAZIANTEP WindSpeed(m/s)'],
      OSMANIYEWindSpeed = row['OSMANIYE WindSpeed(m/s)'],
      CNKWindSpeed = row['CNK WindSpeed(m/s)'],
      MUGLAWindSpeed = row['MUGLA WindSpeed(m/s)'],
      YALOVAWindSpeed = row['YALOVA WindSpeed(m/s)'],
      ADIYAMANWindSpeed = row['ADIYAMAN WindSpeed(m/s)'],
      KOCWindSpeed = row['KOC WindSpeed(m/s)'],
      KAYSERIWindSpeed = row['KAYSERI WindSpeed(m/s)'],
      ISPARTAWindSpeed = row['ISPARTA WindSpeed(m/s)'],
      USAKWindSpeed = row['USAK WindSpeed(m/s)'],
      MERSINWindSpeed = row['MERSIN WindSpeed(m/s)'],
      IZMIRWindSpeed = row['IZMIR WindSpeed(m/s)'],
      MANISAWindSpeed = row['MANISA WindSpeed(m/s)'],
      AFYONWindSpeed = row['AFYON WindSpeed(m/s)']
)
  
  rowsData.append(data)
df_data = pd.DataFrame(rowsData)

In [84]:
df_data.head()

,target,weekDay,weekOfYear,hourOfDay,dayOfMonth,AMASYA_WindSpeed,KIRLARELIWindSpeed,TEKIRDAGWindSpeed,EDIRNEWindSpeed,BURSAWindSpeed,SIVASWindSpeed,BILECIKWindSpeed,AYDINWindSpeed,HATAYWindSpeed,TOKATWindSpeed,KONYAWindSpeed,GAZIANTEPWindSpeed,OSMANIYEWindSpeed,CNKWindSpeed,MUGLAWindSpeed,YALOVAWindSpeed,ADIYAMANWindSpeed,KOCWindSpeed,KAYSERIWindSpeed,ISPARTAWindSpeed,USAKWindSpeed,MERSINWindSpeed,IZMIRWindSpeed,MANISAWindSpeed,AFYONWindSpeed
0,49.25,3,5,0,1,5.175035,3.737460,5.232393,7.066192,0.000000,6.420591,3.083416,5.449154,0.0,2.955127,4.978358,0.0,0.0,0.000000,0.000000,2.593469,0.000000,4.171543,0.0,0.000000,2.652624,5.281608,0.000000,3.320128,1.028135
1,658.52,3,5,1,1,5.175035,5.211629,6.348752,7.404708,4.937590,7.183495,0.000000,6.353604,0.0,3.684031,4.980009,0.0,0.0,1.661267,1.172743,3.740429,0.000000,3.335302,0.0,1.065564,2.397490,5.787553,2.845727,4.239942,1.395394
2,0.00,3,5,2,1,5.175035,6.232947,7.267507,7.958284,6.789526,6.986767,1.829235,8.292643,0.0,4.591133,4.730059,0.0,0.0,3.235681,3.199151,4.434769,0.000000,5.063971,0.0,2.234558,0.000000,5.372688,4.866125,5.598036,1.918763
3,-788.52,3,5,3,1,5.175035,7.760160,8.973306,7.823372,6.220973,4.790768,2.698536,8.849569,0.0,5.501732,4.219697,0.0,0.0,4.567084,5.774808,5.186938,0.000000,5.488671,0.0,2.659959,2.763606,5.787553,5.171029,6.750460,2.251368
4,-497.19,3,5,4,1,5.923936,9.088173,9.807333,9.404032,7.705063,4.527996,3.567999,7.601854,0.0,5.739936,4.156707,0.0,0.0,2.685054,7.068883,6.920417,2.108324,6.464936,0.0,1.065564,3.740429,6.150193,5.919351,8.038033,2.790788


In [85]:
train_size = int(len(df_data) *0.85)
train_df, test_df = df_data[:train_size] , df_data[train_size + 1:]
train_df.shape , test_df.shape

((17523, 30), (3092, 30))

In [86]:
scaler = MinMaxScaler(feature_range=(-1,1))
scaler = scaler.fit(train_df)

train_df = pd.DataFrame(
    scaler.transform(train_df),
    index = train_df.index,
    columns = train_df.columns
)
test_df = pd.DataFrame(
    scaler.transform(test_df),
    index = test_df.index,
    columns = test_df.columns
)

In [94]:
# sequence initialization:

def create_sequence(input_data: pd.DataFrame, target_variable , seq_length):
  sequences = []
  data_size = len(input_data)
  #[1,2,3,5,4,7,6,8]
  for i in tqdm(range(data_size - seq_length)):
    
    sequence = input_data[i:i+seq_length]
    label_position = i+seq_length
    label = input_data.iloc[label_position][target_variable]
    sequences.append((sequence,label))
  return sequences


In [95]:
train_seq = create_sequence(train_df , 'target' , seq_length = 144)

In [96]:
test_seq = create_sequence(test_df , 'target' , seq_length=144)

In [97]:
train_seq[0][0]

,target,weekDay,weekOfYear,hourOfDay,dayOfMonth,AMASYA_WindSpeed,KIRLARELIWindSpeed,TEKIRDAGWindSpeed,EDIRNEWindSpeed,BURSAWindSpeed,SIVASWindSpeed,BILECIKWindSpeed,AYDINWindSpeed,HATAYWindSpeed,TOKATWindSpeed,KONYAWindSpeed,GAZIANTEPWindSpeed,OSMANIYEWindSpeed,CNKWindSpeed,MUGLAWindSpeed,YALOVAWindSpeed,ADIYAMANWindSpeed,KOCWindSpeed,KAYSERIWindSpeed,ISPARTAWindSpeed,USAKWindSpeed,MERSINWindSpeed,IZMIRWindSpeed,MANISAWindSpeed,AFYONWindSpeed
0,0.088306,0.000000,-0.843137,-1.000000,-1.000000,0.045516,-0.312716,-0.001600,0.186810,-1.000000,0.476065,-0.470723,0.017757,-1.000000,-0.457116,0.005025,-1.000000,-1.000000,-1.000000,-1.000000,-0.467559,-1.0,-0.103420,-1.000000,-1.000000,-0.458797,-0.087419,-1.000000,-0.412021,-0.801639
1,0.224874,0.000000,-0.843137,-0.913043,-1.000000,0.045516,-0.041630,0.211414,0.243665,-0.111355,0.651453,-1.000000,0.186684,-1.000000,-0.323210,0.005358,-1.000000,-1.000000,-0.711550,-0.792078,-0.232087,-1.0,-0.283151,-1.000000,-0.771057,-0.510851,0.000000,-0.514714,-0.249126,-0.730783
2,0.077267,0.000000,-0.843137,-0.826087,-1.000000,0.045516,0.146181,0.386723,0.336642,0.221948,0.606226,-0.686006,0.548845,-1.000000,-0.156567,-0.045101,-1.000000,-1.000000,-0.438181,-0.432806,-0.089539,-1.0,0.088387,-1.000000,-0.519892,-1.000000,-0.071682,-0.170173,-0.008614,-0.629808
3,-0.099480,0.000000,-0.843137,-0.739130,-1.000000,0.045516,0.427021,0.712208,0.313983,0.119623,0.101376,-0.536788,0.652864,-1.000000,0.010718,-0.148132,-1.000000,-1.000000,-0.207006,0.023846,0.064882,-1.0,0.179667,-1.000000,-0.428492,-0.436154,0.000000,-0.118178,0.195475,-0.565638
4,-0.034178,0.000000,-0.843137,-0.652174,-1.000000,0.196817,0.671231,0.871350,0.579464,0.386723,0.040966,-0.387543,0.419824,-1.000000,0.054478,-0.160849,-1.000000,-1.000000,-0.533787,0.253280,0.420766,-0.6,0.389493,-1.000000,-0.771057,-0.236857,0.062659,0.009435,0.423498,-0.461567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,0.133910,-0.666667,-0.803922,0.652174,-0.666667,-0.424630,-0.029336,0.103035,-0.289803,-1.000000,0.133410,-0.419597,-1.000000,0.192287,-0.804257,-1.000000,-0.398032,-0.167966,-0.081060,-0.073424,-0.232087,-1.0,0.053479,0.195041,0.135482,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
140,0.103055,-0.666667,-0.803922,0.739130,-0.666667,-1.000000,-0.038765,0.260835,0.024281,-1.000000,0.543889,-0.488780,-0.722655,0.298256,-0.637842,-1.000000,-0.310919,-0.110408,-0.179027,-0.135011,-0.329167,-1.0,0.090636,-0.043656,0.063445,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
141,0.077267,-0.666667,-0.803922,0.826087,-0.666667,-1.000000,0.004489,0.195367,0.400386,-0.345241,0.712708,-0.045337,-0.378553,0.513029,0.082932,-0.873585,-0.241568,0.137547,0.025986,-0.003759,-0.154803,-1.0,0.103938,-0.196313,-0.170542,-1.000000,-1.000000,-1.000000,-1.000000,-0.351149
142,0.077267,-0.666667,-0.803922,0.913043,-0.666667,-1.000000,0.048325,0.121814,0.379789,-1.000000,0.860495,0.098246,0.062659,0.784939,0.456137,-1.000000,-0.398032,0.137547,0.213429,0.162999,-0.467559,-1.0,0.175818,-0.054137,-0.026472,-1.000000,-1.000000,-1.000000,-1.000000,-0.313240
